In [3]:
%cd /content/drive/MyDrive/inference_SAM_video/MedSAM
!pip install -e .

/content/drive/MyDrive/inference_SAM_video/MedSAM
Obtaining file:///content/drive/MyDrive/inference_SAM_video/MedSAM
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_

In [4]:
from segment_anything import sam_model_registry, SamPredictor
medsam = sam_model_registry["vit_b"](checkpoint="/content/drive/MyDrive/inference_SAM_video/ckpt/medsam_vit_b.pth")
medsam.to(device='cuda')
medsampredictor = SamPredictor(medsam)

In [2]:
import numpy as np
def compute_iou(pred_mask, gt_semantic_seg):
    in_mask = np.logical_and(gt_semantic_seg, pred_mask)
    out_mask = np.logical_or(gt_semantic_seg, pred_mask)
    iou = np.sum(in_mask) / np.sum(out_mask)
    return iou
def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum

In [5]:
def repixel_value(arr, is_seg=False):
    if not is_seg:
        min_val = arr.min()
        max_val = arr.max()
        new_arr = (arr - min_val) / (max_val - min_val + 1e-10) * 255.
    return new_arr

In [ ]:
import os
import cv2
import nibabel
import torch
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/Brats/medsam_pointperframe'
datapath = '/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/Brats'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  img3D = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  mask = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  img3d = torch.from_numpy(img3D)
  count = 0
  for j in range(img3D.shape[3]):
      if os.path.exists(os.path.join(resultpath,i,str(j),'dice.npy')):
        continue

      p = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
      if len(p) == 0:
        count += 1
        continue
      print(os.path.join(datapath,i,str(j)))
      input_image = torch.repeat_interleave(img3d, repeats=3, dim=0)
      img2d,gt2d = repixel_value(input_image[:,:,:,j]),repixel_value(mask[:,:,:,j])
      gt2d = gt2d.astype(bool)
      img2d = img2d.permute(1,2,0).numpy()
      medsampredictor.set_image(img2d)
      if count % 1 == 0:
        positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = medsampredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = medsampredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      dice = []
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        dice.append(compute_dice(gt2d,masks[m]))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01315.nii.gz/103
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/0
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/1
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/2
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/3
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/4
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/5
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/6
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/7
/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr/BraTS2021_01307.nii.gz/8
/content/drive/MyDrive/lighting_sam_3d/da

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/polypgen/medsam_pointper10frame'
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  count = 0
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):

      # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
      #   continue
      print(os.path.join(datapath,i,'masks',str(j)))
      img = cv2.imread(os.path.join(datapath,i,'images',str(j) + '.jpg'))
      p = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
      if len(p) == 0:
        continue
      medsampredictor.set_image(img)
      if count % 10 == 0:
        positive_points = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i,str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = medsampredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = medsampredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      gt = np.load(os.path.join(datapath,i,'binarymask',str(j) + '.npy'))
      dice = []
      iou = []
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        dice.append(compute_dice(gt,masks[m]))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/7
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/8
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/9
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/10
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/11
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/12
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/13
/c

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/9'
promptpath = '/content/drive/MyDrive/inference_SAM_video/bbox_prompt'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for i in os.listdir(datapath):
  for j in classes:
    for k in direction:
      count = 0
      for c in range(len(os.listdir(os.path.join(datapath,i,j,k)))):
        # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
        #   continue
        l = str(c) + '.png'
        print(os.path.join(datapath,i,j,k,l))
        img = cv2.imread(os.path.join(datapath,i,j,k,l))
        medsampredictor.set_image(img)
        if count % 300 == 0:
          # positive_points = np.load(os.path.join(promptpath,i,k,str(count),'positive.npy'))
          # negative_points = np.load(os.path.join(promptpath,i,k,str(count),'negative.npy'))

          # input_points = np.concatenate([positive_points, negative_points])
          # for m in range(len(input_points)):
          #   a = input_points[m][0]
          #   input_points[m][0] = input_points[m][1]
          #   input_points[m][1] = a
          # input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
          input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
          masks, _, prev_mask = medsampredictor.predict(
            point_coords=None,
            point_labels=None,
            box = input_box,
            multimask_output=True,
          )
        else:
          masks, _, prev_mask = medsampredictor.predict(
            point_coords=None,
            point_labels=None,
            box = input_box,
            mask_input = prev_mask,
            multimask_output=True,
          )
        count += 1
        gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        dice = []
        iou = []
        os.makedirs(os.path.join(resultpath,i,j,k,l.split('.')[0]),exist_ok=True)
        for m in range(masks.shape[0]):
          np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'mask' + str(m)+'.npy'),masks[m])
          dice.append(compute_dice(gt,masks[m]))
          # iou.append(compute_iou(masks[m],gt))
        d = np.argsort(dice)
        prev_mask = prev_mask[[d[-1]]]
        np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy'),dice)
        # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/99.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/100.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/101.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/102.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/103.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/104.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/105.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/106.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/107.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/108.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/109.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/110.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/111.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/112.png
/content/drive/MyDrive/inference_SAM

In [ ]:
import os
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe'
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for j in classes:
  dice = []
  for k in direction:

    for i in os.listdir(resultpath):
      for l in os.listdir(os.path.join(resultpath,i,j,k)):
        if l == '.ipynb_checkpoints':
          continue
        # print(os.path.join(datapath,i,j,k,l))
        d = np.load(os.path.join(resultpath,i,j,k,l,'dice.npy'))
        dice.append(max(d))
        # IOU.append(max(np.load(os.path.join(resultpath,i,j,k,l,'iou.npy'))))
  print('average dice of ' + j + k + ' is ' + str(np.mean(dice)))
  # print('average iou of ' + j + k + ' is ' + str(np.mean(IOU)))
        # image = cv2.imread(os.path.join(datapath,i,j,k,l + '.png'))
        # mask = np.load(os.path.join(resultpath,i,j,k,l,'mask0.npy'))
        # print(image.shape)

        # gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        # image[gt] = 13
        # image[mask] = 0
        # cv2_imshow(cv2.imread(os.path.join(datapath,i,j,k,l + '.png')))
        # point = np.load(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy'))
        # for p in point:
        #   print(p)
        #   print(gt[p[0]][p[1]])
        #   cv2.circle(image, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
        # cv2_imshow(image)
      # print('average dice of ' + i + j + k + ' is ' + str(np.mean(dice)))
      # print('average iou of ' + i + j + k + ' is ' + str(np.mean(IOU)))

average dice of bg_changeright is 0.5195914112832004
average dice of bloodright is 0.4395995111515578
average dice of low_brightnessright is 0.4609640708596622
average dice of regularright is 0.4335438732313704
average dice of smokeright is 0.43867016078919324


medsam bbox per frame
average dice of bg_changeright is 0.5274602075720326
average dice of bloodright is 0.45356022696133047
average dice of low_brightnessright is 0.48130428358451366
average dice of regularright is 0.44367773505449515
average dice of smokeright is 0.43036717934583923

medsam bbox per 10 frame
average dice of bg_changeright is 0.506908322155892
average dice of bloodright is 0.4394076670561071
average dice of low_brightnessright is 0.46211293388374264
average dice of regularright is 0.4329313026075436
average dice of smokeright is 0.43423242007337004

medsam bbox per 300 frame
average dice of bg_changeright is 0.5195914112832004
average dice of bloodright is 0.4395995111515578
average dice of low_brightnessright is 0.4609640708596622
average dice of regularright is 0.4335438732313704
average dice of smokeright is 0.43867016078919324

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sammed_prompt10perframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/9'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt_for_video_n'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for i in os.listdir(datapath):
  for j in classes:
    for k in direction:
      count = 0
      for c in range(len(os.listdir(os.path.join(datapath,i,j,k)))):
        # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
        #   continue
        l = str(c) + '.png'
        print(os.path.join(datapath,i,j,k,l))
        img = cv2.imread(os.path.join(datapath,i,j,k,l))
        medsampredictor.set_image(img)
        if count % 10 == 0:
          positive_points = np.load(os.path.join(promptpath,i,k,str(count),'positive.npy'))
          negative_points = np.load(os.path.join(promptpath,i,k,str(count),'negative.npy'))

          input_points = np.concatenate([positive_points, negative_points])
          for m in range(len(input_points)):
            a = input_points[m][0]
            input_points[m][0] = input_points[m][1]
            input_points[m][1] = a
          input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])

          masks, _, prev_mask = medsampredictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=True,
          )
        else:
          masks, _, prev_mask = medsampredictor.predict(
            point_coords=None,
            point_labels=None,
            mask_input = prev_mask,
            multimask_output=True,
          )
        count += 1
        gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        dice = []
        iou = []
        os.makedirs(os.path.join(resultpath,i,j,k,l.split('.')[0]),exist_ok=True)
        for m in range(masks.shape[0]):
          np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'mask' + str(m)+'.npy'),masks[m])
          dice.append(compute_dice(gt,masks[m]))
          # iou.append(compute_iou(masks[m],gt))
        d = np.argsort(dice)
        prev_mask = prev_mask[[d[-1]]]
        np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy'),dice)
        # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

In [ ]:
p = '/content/drive/MyDrive/inference_SAM_video/medsam_bboxper300frame'
import numpy as np
import os
for i in ['0','1','2']:
    for j in os.listdir(os.path.join(p,i)):
        for l in ['left','right']:
            print(len(os.listdir(os.path.join(p,i,j,l))))
            for k in os.listdir(os.path.join(p,i,j,l)):
                if not os.path.exists(os.path.join(p,i,j,l,k,'mask.npy')):
                    print(os.path.join(p,i,j,l,k))

300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300


In [ ]:
sam_jointmask_path = '/content/drive/MyDrive/inference_SAM_video/medsam_bboxper300frame'
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
import shutil
import os
import numpy as np
for j in classes:
  for i in os.listdir(resultpath):
    for k in direction:
      for l in range(len(os.listdir(os.path.join(resultpath,i,j,k)))):
        print(os.path.join(resultpath,i,j,k,str(l)))
        dice = np.load(os.path.join(resultpath,i,j,k,str(l),'dice.npy'))
        b = np.argsort(dice)
        os.makedirs(os.path.join(sam_jointmask_path,i,j,k,str(l)),exist_ok=True)
        shutil.move(os.path.join(resultpath,i,j,k,str(l),'mask' + str(b[-1]) + '.npy'),os.path.join(sam_jointmask_path,i,j,k,str(l),'mask.npy'))

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/100
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/101
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/102
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/103
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/104
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/105
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/106
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/107
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/2/low_brightness/right/108
/content/drive/MyDrive/inference_SAM_video/result_medsam_bbox300perframe/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
